In [1]:
import numpy as np
import pandas as pd

In [2]:
# Mean Squared Error loss
def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

In [3]:
# Load dataset
data = data = pd.read_csv('./housing.csv')

In [4]:
# Print The head of the data 
data.head()

,RM,LSTAT,PTRATIO,MEDV
0,6.575,4.98,15.3,504000.0
1,6.421,9.14,17.8,453600.0
2,7.185,4.03,17.8,728700.0
3,6.998,2.94,18.7,701400.0
4,7.147,5.33,18.7,760200.0


In [5]:
#Printing the number of NULL elements
data.isnull().sum()

RM         0
LSTAT      0
PTRATIO    0
MEDV       0
dtype: int64

In [6]:
#Remove Row in Rows with NULL 
data = data.dropna()

In [7]:
#Describe the Data
data.describe()

,RM,LSTAT,PTRATIO,MEDV
count,489.000000,489.000000,489.000000,4.890000e+02
mean,6.240288,12.939632,18.516564,4.543429e+05
std,0.643650,7.081990,2.111268,1.653403e+05
min,3.561000,1.980000,12.600000,1.050000e+05
25%,5.880000,7.370000,17.400000,3.507000e+05
50%,6.185000,11.690000,19.100000,4.389000e+05
75%,6.575000,17.120000,20.200000,5.187000e+05
max,8.398000,37.970000,22.000000,1.024800e+06


In [8]:
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def r_squared(y_true, y_pred):
    ss_total = np.sum((y_true - np.mean(y_true)) ** 2)
    ss_residual = np.sum((y_true - y_pred) ** 2)
    return 1 - (ss_residual / ss_total)

In [9]:
class NN:
    def __init__(self, input_layer_size, hidden_layer_size, output_layer_size, lr, epoches=1000):  # constructor with flexible inputs
        self.input_layer_size = input_layer_size
        self.hidden_layer_size = hidden_layer_size
        self.output_layer_size = output_layer_size
        self.lr = lr
        self.number_of_epoches = epoches

        # Initialize the weights for input-to-hidden and hidden-to-output layers
        self.W1 = np.random.randn(self.input_layer_size, self.hidden_layer_size)
        self.W2 = np.random.randn(self.hidden_layer_size, self.output_layer_size)

        # Bias terms (optional took values from example provided)
        self.b1 = 0.40
        self.b2 = 0.80

    # Activation function (sigmoid)
    def activation(self, x):
        return 1 / (1 + np.exp(-x))

    # Forward pass
    def forward(self, X):
        self.hidden = self.activation(np.dot(X, self.W1) + self.b1)  # H = activation(X * W1 + b1)
        self.output = self.activation(np.dot(self.hidden, self.W2) + self.b2)  # Output = activation(H * W2 + b2)
        return self.output

    # Backpropagation
    def back_prop(self, X, y):
        output = self.forward(X)
        # Output layer -> Hidden layer weight update (W2)
        delta_output = -(y - output) * output * (1 - output)  # Output layer error gradient
        self.W2 -= self.lr * np.dot(self.hidden.T, delta_output)  # Update W2

        # Hidden layer -> Input layer weight update (W1)
        delta_hidden = np.dot(delta_output, self.W2.T) * self.hidden * (1 - self.hidden)  # Hidden layer error gradient
        self.W1 -= self.lr * np.dot(X.T, delta_hidden)  # Update W1

    # Training the network
    def train(self, X, y):
        for epoch in range(self.number_of_epoches):
            self.back_prop(X, y)
            # Optional: Calculate and print the loss after each epoch
            # output = self.forward(X)
            # loss = mean_squared_error(y, output)
            # print(f'Epoch {epoch + 1}/{self.number_of_epoches}, Loss: {loss}')

    # Prediction
    def predict(self, X):
        return self.forward(X)

In [10]:
def cross_validation(X, y, k, model_class, **kwargs):
    fold_size = len(X) // k
    indices = np.arange(len(X))
    np.random.shuffle(indices)

    mse_scores = []
    r2_scores = []

    for i in range(k):
        # Define the validation and training set
        val_indices = indices[i * fold_size:(i + 1) * fold_size]
        train_indices = np.concatenate((indices[:i * fold_size], indices[(i + 1) * fold_size:]))

        X_train, X_val = X[train_indices], X[val_indices]
        y_train, y_val = y[train_indices], y[val_indices]

        # Create the model and train it
        model = model_class(X_train.shape[1], **kwargs)
        model.train(X_train, y_train)

        # Predict and evaluate on the validation set
        y_pred = model.predict(X_val)
        mse_scores.append(mean_squared_error(y_val, y_pred))
        r2_scores.append(r_squared(y_val, y_pred))
    y_pred = model.predict(X)
    return mse_scores[-1],r2_scores[-1]

In [13]:
# Get user inputs for model configuration
input_neurons = int(input("Enter the number of neurons in the input layer: "))
output_neurons = int(input("Enter the number of neurons in the output layer: "))
epoches = int(input("Enter the number of epochs: "))
hidden_neurons = int(input("Enter the number of neurons in the hidden layer: "))
learning_rate = float(input("Enter the learning rate: "))

# Load your dataset (assuming df is a pandas DataFrame)
# df will have features and targets, we select them dynamically based on user input
X = data.iloc[:, :input_neurons].values  # Selecting the first 'input_neurons' columns as features
y = data.iloc[:, -output_neurons:].values  # Selecting the last 'output_neurons' columns as targets
X = (X - np.min(X)) / (np.max(X) - np.min(X))
y = (y - np.min(y)) / (np.max(y) - np.min(y))

# Perform 5-Fold Cross Validation
print("Performing 5-Fold Cross Validation : ")
mse_5, r2_5 = cross_validation(X, y, 5, NN, hidden_layer_size=hidden_neurons, output_layer_size=output_neurons, lr=learning_rate, epoches=epoches)
print(f"5-Fold Cross Validation MSE: {mse_5}, R-squared: {r2_5}")

# Perform 10-Fold Cross Validation
print("Performing 10-Fold Cross Validation : ")
mse_10, r2_10 = cross_validation(X, y, 10, NN, hidden_layer_size=hidden_neurons, output_layer_size=output_neurons, lr=learning_rate, epoches=epoches)
print(f"10-Fold Cross Validation MSE: {mse_10}, R-squared: {r2_10}")

Enter the number of neurons in the input layer:  3
Enter the number of neurons in the output layer:  1
Enter the number of epochs:  1000
Enter the number of neurons in the hidden layer:  5
Enter the learning rate:  0.0001


Performing 5-Fold Cross Validation : 
5-Fold Cross Validation MSE: 0.029463369780043294, R-squared: 0.3622072178989698
Performing 10-Fold Cross Validation : 
10-Fold Cross Validation MSE: 0.041079637934789213, R-squared: 0.10733465392108144


## MY OUTPUTS

Enter the number of neurons in the hidden layer:  5
Enter the learning rate:  0.0001
Performing 5-Fold Cross Validation : 
5-Fold Cross Validation MSE: 0.029463369780043294, R-squared: 0.3622072178989698
Performing 10-Fold Cross Validation : 
10-Fold Cross Validation MSE: 0.041079637934789213, R-squared: 0.10733465392108144